In [54]:
import pandas as pd
import numpy as np

# readin data
df = pd.read_excel('data\Database-Welcome.xlsx')
# looking at the column in interest
df.iloc[:, 7]

0                  Python Julia
1                  Python T-Sql
2                   Python R JS
3                     Python JS
4                 Python CSharp
5     Python C_sharp Javascript
6                      Python R
7           Javascript Python R
8                   python r js
9                      python R
10                       Python
11                          NaN
12                     Python R
13                R Python Java
14          Python R Javascript
15            Python Javascript
16                          NaN
17                       python
18                          NaN
Name: Which programming languages do you know?:, dtype: object

In [55]:
# getting (almost) unique entries for the known languages
p_lang_entries = [[*i.split(' ')] for i in df.iloc[:, 7].fillna('None')]
p_langs = pd.Series([x for l in p_lang_entries for x in l]).str.lower()
p_langs.value_counts()

python        16
r              8
javascript     4
js             3
none           3
julia          1
t-sql          1
csharp         1
c_sharp        1
java           1
dtype: int64

In [38]:
from fuzzywuzzy import fuzz

# creating a fuzz-ratio matrix of already known languages
lang_fuzz = pd.DataFrame(columns = p_langs.unique(), index = p_langs.unique()).fillna(0)

# filling the matrix with the fuzzy-ratio values
for i, col in lang_fuzz.iteritems():
    for j, row in col.iteritems():
        lang_fuzz.loc[i, j] = fuzz.ratio(i, j)

lang_fuzz

Writing the function

In [53]:
languages = [
    "python", "csharp", "julia", 
    "none", "java", "javascript", 
    "r", "scala", "matlab", "go", 
    "c", "c++", "ruby", "perl", "py",
    "php", "swift", "haskell", "js",
    "clojure", "erlang", "lua", 
    "rust", "d", "dart", "ocaml", 
    "prolog", "f#", "fortran", 
    "vb.net", "visual_basic", "sql", # gonna treat t-sql, postgresql, mysql as the same
    "assembly", "erl", "elixir", "elm"
    ]

def match_lang(x, threshold = 74):
    """Matches a string to a given programming language. If it fails 
    to do so, it creates a new entry in the list of languages"""
    
    # making it lowercase
    lower_x = x.lower()
    # creating a dict that will store the fuzzy ratios
    scores = {i:0 for i in languages}
    
    # checking fuzzy ratio for each registered language
    for lang in languages:
        scores[lang] = fuzz.ratio(lang, lower_x)
    
    # print("Best match:", max(scores, key=scores.get), max(scores.values()))
    
    if max(scores.values()) < threshold:
        print("Adding new language:", x)
        languages.append(x)
        
        return x
    else:
        # get best match
        best_x = max(scores, key=scores.get)
        
        # check if it is a known short form
        if best_x == "js":
            return "javascript"
        elif best_x == "py":
            return "python"
        
        # return the best match
        return best_x

In [52]:
match_lang("r")

r 100
